# AB Test Course - PA James 

## 0.1 Imports

In [65]:
import pandas as pd
import math
from statsmodels.stats import api as sms
import numpy as np

# 1.0 Load data

In [58]:
df_raw = pd.read_csv('../datasets/ab_data.csv')

In [59]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# 2.0 Design de Experimentos

## 2.1 Formulação da Hipóteses

## 2.2 Parâmetros do Experimento

In [63]:
#Nivel de confiança
confidence_level = 0.95

#Nivel de significancia 
significance_level = 0.05


p1=0.13
p2=0.15

# tamanho do efeito
effect_size = sms.proportion_effectsize(p1,p2)

#Power estatistico
power = 0.8


In [68]:
#sample size
sample_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=power,
    alpha=significance_level)

sample_n = math.ceil(sample_n)
sample_n

In [70]:
print('O tamanho total da amostra {}'.format(2*sample_n))
print('O tamanho da amostra do grupo controle é de {}'.format(sample_n))
print('O tamanho da amostra do grupo de tratamento é de {}'.format(sample_n))


O tamanho total da amostra 9440
O tamanho da amostra do grupo controle é de 4720
O tamanho da amostra do grupo de tratamento é de 4720


In [77]:
conversion_rate = 0.10 #quantas vão abrir/responder
x= sample_n / conversion_rate
print('o numero de email para enviar:{}'.format(int(x)))
print('o numero total da base de emial para enviar:{}'.format(2*int(x)))


o numero de email para enviar:47200
o numero total da base de emial para enviar:94400


In [ ]:
#Total de pessoas para o estudo com esse parametos são muito alto.
#Podemos aumentar as porcentagem de converções o que não seria ideal, pois uma diferença da p de tratmento deve ser o menor possivel, assim não tendo diferença na aleatoriedade.
